In [13]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import PromptTemplate
from transformers import BitsAndBytesConfig

In [8]:
docs = SimpleDirectoryReader("../../data/uploads").load_data()
docs

[Document(id_='3ec66007-6677-4be1-86c1-6c1a25d370e2', embedding=None, metadata={'page_label': '1', 'file_name': '1301.3781v3.pdf', 'file_path': '/home/vedaya/git/research_copilot/research_copilot/core/../../data/uploads/1301.3781v3.pdf', 'file_type': 'application/pdf', 'file_size': 228716, 'creation_date': '2024-12-21', 'last_modified_date': '2024-12-12'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Efﬁcient Estimation of Word Representations in\nVector Space\nTomas Mikolov\nGoogle Inc., Mountain View, CA\ntmikolov@google.com\nKai Chen\nGoogle Inc., Mountain View, CA\nkaichen@google.com\nGreg Corrado\nGoogle Inc., Mountain View, CA\ngc

In [10]:
system_prompt = """
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""
template = (
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given this information, please answer the question: {query_str}\n"
)
qa_template = PromptTemplate(template)

In [14]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [16]:
import torch
llm = HuggingFaceLLM(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    tokenizer_name="sentence-transformers/all-MiniLM-L6-v2",
    context_window=512,
    max_new_tokens=256,
    model_kwargs={"quantization_config": quantization_config},
    generate_kwargs={"temperature": 0.7, "top_k": 50, "top_p": 0.95},
    system_prompt=system_prompt,
    device_map="cuda:0",
)

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of BertLMHeadModel were not initialized from the model checkpoint at sentence-transformers/all-MiniLM-L6-v2 and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

from llama_index.legacy.service_context import ServiceContext
from llama_index.embeddings.langchain import LangchainEmbedding


In [28]:
embed_model=LangchainEmbedding(
    HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2"))
service_context=ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)
service_context

ServiceContext(llm_predictor=LLMPredictor(system_prompt=None, query_wrapper_prompt=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>), prompt_helper=PromptHelper(context_window=512, num_output=256, chunk_overlap_ratio=0.1, chunk_size_limit=None, separator=' '), embed_model=LangchainEmbedding(model_name='sentence-transformers/all-mpnet-base-v2', embed_batch_size=10, callback_manager=<llama_index.legacy.callbacks.base.CallbackManager object at 0x764a0c40f3e0>, num_workers=None), transformations=[SentenceSplitter(include_metadata=True, include_prev_next_rel=True, callback_manager=<llama_index.legacy.callbacks.base.CallbackManager object at 0x764a0c40f3e0>, id_func=<function default_id_func at 0x764a25f17ec0>, chunk_size=1024, chunk_overlap=200, separator=' ', paragraph_separator='\n\n\n', secondary_chunking_regex='[^,.;。？！]+[,.;。？！]?')], llama_logger=<llama_index.legacy.logger.base.LlamaLogger object at 0x764a0c317f20>, callback_manager=<llama_index.legacy.callbacks.bas

In [ ]:
index = VectorStoreIndex.from_documents(docd, service_context=service_context)
query_engine=index.as_query_engine()
     
response=query_engine.query("what is attention is all you need?")